<a href="https://colab.research.google.com/github/glukonatic/III/blob/master/13Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Импорт библиотек

import numpy as np 

import re 

from tensorflow.keras.models import Model, load_model 
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input 
from tensorflow.keras.optimizers import RMSprop, Adadelta 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import utils 
from tensorflow.keras.utils import plot_model 

import yaml 

In [2]:
#@title Подключаем гуглдрайв

from google.colab import files 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#@title Класс для ДЗ

class HW13():

  def __init__(self):
    corpuspath = '/content/drive/My Drive/III/13/Диалоги(рассказы).yml'
    corpus = open(corpuspath, 'r') 
    document = yaml.safe_load(corpus) 
    self.conversations = document['разговоры']
    self.questions = list()
    self.answers = list() 
    self.results = list()
    print('Класс инициализирован.')

  def prepare(self, tknzr_type = 'simple'):

    self.tokenizer = Tokenizer()

    for con in self.conversations: 
      if type(con[1]) == str:
        self.questions.append(con[0]) 
        self.answers.append('<START> ' + (' ').join(con[1:]) + ' <END>') 

    if tknzr_type == 'with_punctuation':
      to_tokenize = '.,!?'
      self.questions = list(map(lambda l: re.sub(r'(['+to_tokenize+'])', r' \1 ', str(l)), self.questions))
      self.answers = list(map(lambda l: re.sub(r'(['+to_tokenize+'])', r' \1 ', str(l)), self.answers))

    self.tokenizer.fit_on_texts(self.questions + self.answers) 
    vocabularyItems = list(self.tokenizer.word_index.items()) 
    self.vocabularySize = len(vocabularyItems)+1 

    tokenizedQuestions = self.tokenizer.texts_to_sequences(self.questions) 
    self.maxLenQuestions = max([ len(x) for x in tokenizedQuestions]) 
    paddedQuestions = pad_sequences(tokenizedQuestions, maxlen=self.maxLenQuestions, padding='post')
    self.encoderForInput = np.array(paddedQuestions) 

    tokenizedAnswers = self.tokenizer.texts_to_sequences(self.answers) 
    self.maxLenAnswers = max([len(x) for x in tokenizedAnswers]) 
    paddedAnswers = pad_sequences(tokenizedAnswers, maxlen=self.maxLenAnswers, padding='post')
    self.decoderForInput = np.array(paddedAnswers) 

    for i in range(len(tokenizedAnswers)) : 
      tokenizedAnswers[i] = tokenizedAnswers[i][1:] 
    paddedAnswers = pad_sequences(tokenizedAnswers, maxlen=self.maxLenAnswers , padding='post')
    self.oneHotAnswers = utils.to_categorical(paddedAnswers, self.vocabularySize) 
    self.decoderForOutput = np.array(self.oneHotAnswers) 
    print('Данные подготовлены.')
  
  def create_nn(self):

    self.encoderInputs = Input(shape=(11, )) 
    encoderEmbedding = Embedding(self.vocabularySize, 200,  mask_zero=True) (self.encoderInputs)
    encoderOutputs, state_h , state_c = LSTM(200, return_state=True)(encoderEmbedding)
    self.encoderStates = [state_h, state_c]

    self.decoderInputs = Input(shape=(13, )) 
    self.decoderEmbedding = Embedding(self.vocabularySize, 200, mask_zero=True) (self.decoderInputs) 
    self.decoderLSTM = LSTM(200, return_state=True, return_sequences=True)
    decoderOutputs , _ , _ = self.decoderLSTM (self.decoderEmbedding, initial_state=self.encoderStates)
    self.decoderDense = Dense(self.vocabularySize, activation='softmax') 
    output = self.decoderDense (decoderOutputs)

    self.model = Model([self.encoderInputs, self.decoderInputs], output)
    self.model.compile(optimizer=RMSprop(), loss='categorical_crossentropy')
    print('Модель скомпилирована.')
    
  def fit(self, bs=50, ep=10):

    print('Запуск обучения...') 
    self.model.fit([self.encoderForInput , self.decoderForInput], 
                   self.decoderForOutput, 
                   batch_size=bs, epochs=ep, verbose = 0) 
    print('Цикл обучения завершен')

  def makeInferenceModels(self):
  
    encoderModel = Model(self.encoderInputs, self.encoderStates) 

    decoderStateInput_h = Input(shape=(200 ,)) 
    decoderStateInput_c = Input(shape=(200 ,))

    decoderStatesInputs = [decoderStateInput_h, decoderStateInput_c] 

    decoderOutputs, state_h, state_c = self.decoderLSTM(self.decoderEmbedding, initial_state=decoderStatesInputs)

    decoderStates = [state_h, state_c] 
    decoderOutputs = self.decoderDense(decoderOutputs) 

    decoderModel = Model([self.decoderInputs] + decoderStatesInputs, [decoderOutputs] + decoderStates)

    return encoderModel , decoderModel


  def strToTokens(self, sentence: str): 

    words = sentence.lower().split() 
    tokensList = list() 

    for word in words: 
      tokensList.append(self.tokenizer.word_index[word]) if word in self.tokenizer.word_docs.keys() else tokensList.append(1) 

    return pad_sequences([tokensList], maxlen=self.maxLenQuestions , padding='post')


  def main(self, dialog_type = 'manual'):
    
    encModel, decModel = self.makeInferenceModels() 

    self.auto_phrases = ['Привет','Как дела','Почему так','Ты вообще кто','А если найду','Ужас какой']

    for i in range(6): 

      if dialog_type == 'auto':
        statesValues = encModel.predict(self.strToTokens(self.auto_phrases[i]))
      else:
        statesValues = encModel.predict(self.strToTokens(input( 'Задайте вопрос : ' )))


      emptyTargetSeq = np.zeros((1, 1))    
      emptyTargetSeq[0, 0] = self.tokenizer.word_index['start'] 

      stopCondition = False 
      decodedTranslation = '' 

      while not stopCondition : 

        decOutputs , h , c = decModel.predict([emptyTargetSeq] + statesValues)
      
        sampledWordIndex = np.argmax(decOutputs, axis=-1) 
        sampledWord = None 
        for word , index in self.tokenizer.word_index.items():
          if sampledWordIndex == index: 
            decodedTranslation += ' {}'.format(word) 
            sampledWord = word 
      
        if sampledWord == 'end' or len(decodedTranslation.split()) > self.maxLenAnswers:
          stopCondition = True 

        emptyTargetSeq = np.zeros((1, 1)) 
        emptyTargetSeq[0, 0] = sampledWordIndex 
        statesValues = [h, c] 

      if dialog_type == 'auto':
        self.results.append(decodedTranslation[:-3]) 
      else:
        print(decodedTranslation[:-3]) 

  def clear_vars(self):
    
    self.questions.clear()
    self.answers.clear()
    del self.tokenizer
    del self.vocabularySize
    del self.maxLenQuestions
    del self.encoderForInput
    del self.maxLenAnswers
    del self.decoderForInput
    del self.oneHotAnswers
    del self.decoderForOutput

    print('Переменные удалены и очищены')

In [4]:
#@title Главный блок

#@markdown Количество циклов обучения по 10 эпох:
cycles =  10#@param {type: "number"}

first = HW13()

first.prepare()
first.create_nn()

for i in range(cycles):
  first.fit()
  first.main('auto')

first.clear_vars()

second = HW13()

second.prepare('with_punctuation')
second.create_nn()

for i in range(cycles):
  second.fit()
  second.main('auto')

second.clear_vars()

Класс инициализирован.
Данные подготовлены.
Модель скомпилирована.
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Переменные удалены и очищены
Класс инициализирован.
Данные подготовлены.
Модель скомпилирована.
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения завершен
Запуск обучения...
Цикл обучения заверш

In [6]:
print(first.results)
print(second.results)

[' да ', ' да ', ' не знаю ', ' нет нет ', ' да ', ' да ', ' что случилось ', ' да товарищ командир ', ' не знаю ', ' нет я не знаю а что ', ' как же вы знаете ', ' а где ', ' ну как что вы с вами делать ', ' а почему бы и нет ', ' потому что у меня есть есть ', ' я не знаю я а ты мне не об этом этом ', ' как мы сами и с тобой и я ', ' а где ', ' что ты здесь делаешь ', ' все в порядке ', ' потому что у вас там нет ', ' меня я должен знать сейчас пойду ты ', ' как мы с вами все и в порядке ', ' ну на что нибудь ', ' что ты здесь делаешь ', ' вот так ', ' ты так просто ', ' меня я пойду один сейчас ты меня не умею меня ', ' ну и сказала все и его к нет ', ' на кто нибудь из чего ', ' что ты здесь делаешь ', ' за вами генерал ', ' потому что у вас есть они ', ' а я конечно не сколько ', ' ну и сказала я тоже не знаю ', ' ну на что нибудь ', ' что тебе надо человек ', ' за вами ну вот так ', ' потому что у вас есть ', ' а я конечно не сказала что ли кто не его в дело ', ' мы будем из тебя